In [24]:
%pip install requests -q
%pip install pandas -q
%pip install beautifulsoup4 -q
%pip install tqdm -q


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
import requests

def fetch_sac_program_calendar(
    year=2025, month=8, first_day=1, last_day=31, category_primary=2
):
    url = "https://www.sac.or.kr/site/main/program/getProgramCalList"
    # 서버에서 expect application/json response
    payload = {
        "searchYear": str(year),
        "searchMonth": f"{month:02d}",
        "searchFirstDay": str(first_day),
        "searchLastDay": str(last_day),
        "CATEGORY_PRIMARY": str(category_primary)
    }

    headers = {
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        # Optional: 필요한 경우 Referer, User-Agent 등 추가
        "User-Agent": "python-requests/2.x"
    }

    resp = requests.post(url, data=payload, headers=headers)
    resp.raise_for_status()  # 오류 처리

    try:
        result = resp.json()
    except ValueError:
        print("JSON 파싱 실패, 원본 응답:", resp.text)
        raise

    return result

data = fetch_sac_program_calendar()

In [40]:
import pandas as pd

flattened = []

for day, events in data.items():
    if day == 'result':
        continue
    for event in events:
        event_copy = event.copy()
        event_copy["day"] = day  # 날짜 정보를 새로운 컬럼으로 추가
        flattened.append(event_copy)

# STEP 3: DataFrame으로 변환
df = pd.DataFrame(flattened)
df.head().T

,0,1,2,3,4
update_id,bahn27,800680000001,NaN,NaN,801082000013
HOMEPAGE_OPEN_FLAG,1,1,1,1,1
TICKET_OPEN_FLAG,N,Y,N,N,Y
CATEGORY_SECONDARY_NAME_ENG,Classical Music,Classical Music,Classical Music,Classical Music,NaN
PROGRAM_SUBJECT_ENG,RENDEZ-VOUS DE MUSIQUE FESTIVAL 2025 (08.22),2025 Hic et Nunc! Music Festival - JIJI Guitar...,Jieun Lee Cello Recital,Pianist Daisy Solo Recital,Kwangchul Youn X Samuel Youn X Gihoon Kim <Sin...
EXPOSE_DATE,,NaN,NaN,NaN,NaN
END_DATE,2025.08.22,2025.08.22,2025.08.22,2025.08.22,2025.08.23
SALE_STATE_CODE_NAME_ENG,Book tickets,Book tickets,Book tickets,Ready,Book tickets
PRICE_INFO_ENG,"R 50,000won / S 30,000won","R 50,000won / S 30,000won","General seats 20,000won","General seats 20,000won","R 132,000won / S 121,000won / A 110,000won / B..."
SAC_PRE_SALE_YN,N,Y,N,N,Y


In [41]:
import requests
from bs4 import BeautifulSoup

def get_detail(sn, logging=False):
    url = f"https://www.sac.or.kr/site/main/show/show_view?SN={sn}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # 첫 번째 탭 내 li 항목들
    first_tab = soup.find(class_="cwa-tab")
    if not first_tab:
        return None

    li_items = first_tab.find_all("li")
    li_texts = [li.get_text(strip=True) for li in li_items]

    container = soup.find(class_="cwa-tab-list")
    tabs = container.find_all("div", class_="ctl-sub")

    if logging:
        print(li_texts)

        for t in [tab.get_text(strip=True) for tab in tabs]:
            print(t)
            print('----------')

    # '작품소개' 항목의 인덱스 찾기
    try:
        intro_index = li_texts.index("작품소개")
    except ValueError:
        return None  # '작품소개' 항목이 없으면 None 반환

    # 전체 탭 컨테이너에서 해당 인덱스의 컨텐츠 탭 가져오기
    if not container:
        return None
    
    if intro_index >= len(tabs):
        return None
    return tabs[intro_index].prettify(), tabs[intro_index].get_text(strip=True)

for sn in df['SN'][:10]:
    print(sn)
    
    detail = get_detail(sn, logging=True)
    print(detail)

    print('=======================================')
    


75383
['알립니다', '작품소개', '할인정보', '예매통계', '장소안내', '관람평(0)', '예매안내']
알립니다.[일시적 티켓 판매 중단 및 공연시간 변경 안내]공연장 내부사정으로 인해 판매를 일시적으로 중단합니다.해당 공연은 추후 오후 7시 30분으로 공연 시간이 변경되어 재오픈될 예정입니다.관객 여러분의 너른 양해부탁드립니다. 감사합니다.작품소개 바로가기
----------
PROGRAMWitold Lutoslawski – Variations on a Theme
  by Paganini for Two Pianos비톨드 루토스와프스키–두 대의 피아노를 위한‘파가니니 주제에 의한 변주곡’Pf.원재연,황건영Luigi Boccherini -  String
  Quintet No.60 G324 'Night Music of the Streets of Madrid'루이지 보케리니–현악 오중주 제60번G324 '마드리드 거리의 밤 음악'Ⅰ. Le campane dell'Ave MariaⅡ. Il tamburo dei Soldatilll. Minuetto dei CeichilV. Il RosarioV. Passa CalleVI. Il tamburoVII. RitirataVn.송지원,정주은Va.임지환Vc.채훈선,정우찬Ludwig Thuille - Sextet for Piano and Woodwind Quintet in B♭ Major, Op.6루드비히 투일레–피아노와 목관 오중주를 위한 육중주B♭장조, Op.6I.
  Allegro moderatoII.
  LarghettoIII.
  Gavotte. Andante, quasi AllegrettoIV.
  Finale. VivacePf.김혜진Fl.유채연Ob.윤성영Cl.김상윤Fg.이민호Hr. 미샤 에마노브스키INTERMISSIONJudd Greenstein - City Boy *국내초연주드 그린스타인–시티 보이Pf.황건영Fl.유채연Cl.김상윤Db.임채문Elec Guitar. 김영민Sergei
  Rachmanino

In [42]:
from tqdm import tqdm

results = [get_detail(sn) for sn in tqdm(df["SN"])]

df["detail_html"] = [r[0] if r is not None else None for r in results]
df["detail_text"] = [r[1] if r is not None else None for r in results]


100%|██████████| 94/94 [00:44<00:00,  2.10it/s]


In [43]:
df.drop(columns=['detail_html']).to_csv('result.csv', encoding='utf8')

In [44]:
df['PROGRAM_SUBJECT'].to_list()

['랑데뷰 드 라 무지크 페스티벌 2025(08.22)',
 '2025 힉엣눙크! 뮤직 페스티벌 - 기타리스트 지지 리사이틀 ',
 '이지은 귀국 첼로 독주회',
 '피아니스트 데이지 피아노 독주회',
 '연광철 X 사무엘 윤 X 김기훈 <싱 로우 앤 소프트>',
 '청소년을 위한 인문학 콘서트: 루브르',
 '2025 서울시향 실내악 시리즈 IV: 윤 젱',
 '이찬미 귀국 반주 리사이틀',
 '홍진선 비올라 독주회',
 '쇼팽, 리스트 그리고 파가니니',
 '문록선 해설과 함께하는  제10회 플루트앙상블 아디나 정기연주회',
 '트리오 베스페로 정기연주회',
 '한솔 귀국 플루트 & 피콜로 독주회',
 '2025 힉엣눙크! 뮤직 페스티벌 - 세종솔로이스츠와 길 샤함 & 아델 앤서니 <슬퍼할 때와 춤출 때>',
 '김승수 & 안평강 타악기 듀오 리사이틀',
 '이은준 플루트 독주회 - Origin of French Flute School: Souvenir de France II',
 '2025 온드림 아티스트 시리즈 I : 비올리스트 유혜림',
 '2025 힉엣눙크! 뮤직 페스티벌 - 베르나르 베르베르 x 세종솔로이스츠 <키메라의 시대>',
 '함지민 바이올린 독주회',
 '지유경 피아노 독주회',
 '2025 온드림 아티스트 시리즈 II : 베이스바리톤 김재율',
 '스미노 하야토 피아노 리사이틀',
 '조숙현 피아노 독주회',
 '라 플루티시모 정기연주회',
 '프라이탁 리트 듀오 콘서트',
 'KBS교향악단 제817회 정기연주회',
 '아우어퍼쿠션 퍼커시브 시리즈 I - 프랑스',
 '승경훈 플루트 독주회',
 '유예지 피아노 독주회',
 '디즈니 판타지아 인 콘서트',
 '알테무지크서울 제25회 정기연주회',
 '트리오 인',
 '이건용 트롬본 독주회',
 '나재령 플루트 독주회',
 '한가현 김로운 조인트 리사이틀',
 '히사이시 조 영화음악 콘서트 2025_서울(예술의전당)',
 '이디오마 델라 무지카 정기연주회',
 '피아니스트 이재혁 실내악 시